In [5]:
import settings
import mysql.connector
import pandas as pd
import time
import itertools
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
import plotly.express as px
import datetime
from IPython.display import clear_output

import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# py.init_notebook_mode()
    
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import sqlite3


STATES = ['Alabama', 'AL', 'Alaska', 'AK', 'American Samoa', 'AS', 'Arizona', 'AZ', 'Arkansas', 'AR', 'California', 'CA', 'Colorado', 'CO', 'Connecticut', 'CT', 'Delaware', 'DE', 'District of Columbia', 'DC', 'Federated States of Micronesia', 'FM', 'Florida', 'FL', 'Georgia', 'GA', 'Guam', 'GU', 'Hawaii', 'HI', 'Idaho', 'ID', 'Illinois', 'IL', 'Indiana', 'IN', 'Iowa', 'IA', 'Kansas', 'KS', 'Kentucky', 'KY', 'Louisiana', 'LA', 'Maine', 'ME', 'Marshall Islands', 'MH', 'Maryland', 'MD', 'Massachusetts', 'MA', 'Michigan', 'MI', 'Minnesota', 'MN', 'Mississippi', 'MS', 'Missouri', 'MO', 'Montana', 'MT', 'Nebraska', 'NE', 'Nevada', 'NV', 'New Hampshire', 'NH', 'New Jersey', 'NJ', 'New Mexico', 'NM', 'New York', 'NY', 'North Carolina', 'NC', 'North Dakota', 'ND', 'Northern Mariana Islands', 'MP', 'Ohio', 'OH', 'Oklahoma', 'OK', 'Oregon', 'OR', 'Palau', 'PW', 'Pennsylvania', 'PA', 'Puerto Rico', 'PR', 'Rhode Island', 'RI', 'South Carolina', 'SC', 'South Dakota', 'SD', 'Tennessee', 'TN', 'Texas', 'TX', 'Utah', 'UT', 'Vermont', 'VT', 'Virgin Islands', 'VI', 'Virginia', 'VA', 'Washington', 'WA', 'West Virginia', 'WV', 'Wisconsin', 'WI', 'Wyoming', 'WY']
STATE_DICT = dict(itertools.zip_longest(*[iter(STATES)] * 2, fillvalue=""))
INV_STATE_DICT = dict((v,k) for k,v in STATE_DICT.items())

ImportError: DLL load failed: The specified module could not be found.

In [10]:
conn1 = sqlite3.connect('twitter.db')
print("Opened the database successfully")

conn = conn1.cursor()

timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=20)).strftime('%Y-%m-%d %H:%M:%S')

query = "SELECT id_str, text, created_at, polarity, named_ent ,user_location FROM {} WHERE created_at <= '{}' " .format(settings.TABLE_NAME, timenow)
df = pd.read_sql(query, con=conn1)
df['created_at'] = pd.to_datetime(df['created_at'])

fig = make_subplots(
        rows=2, cols=2,
        column_widths=[1, 0.4],
        row_heights=[0.6, 0.4],
        specs=[[{"type": "scatter", "rowspan": 2}, {"type": "choropleth"}],
               [            None                    , {"type": "bar"}]]
        )

    
    
'''
Plot the Line Chart
'''
# Clean and transform data to enable time series
result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
result = result.rename(columns={"id_str": "Num of '{}' mentions".format(settings.TRACK_WORDS[0]), "created_at":"Time in UTC"})  
time_series = result["Time in UTC"][result['polarity']==0].reset_index(drop=True)
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==0].reset_index(drop=True),
    name="Neural",
    opacity=0.8), row=1, col=1)   
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==-1].reset_index(drop=True),
    name="Negative",
    opacity=0.8), row=1, col=1)
fig.add_trace(go.Scatter(
    x=time_series,
    y=result["Num of '{}' mentions".format(settings.TRACK_WORDS[0])][result['polarity']==1].reset_index(drop=True),
    name="Positive",
    opacity=0.8), row=1, col=1)

'''
Plot the Bar Chart
'''
# content = ' '.join(df["text"])
# content = re.sub(r"http\S+", "", content)
# content = content.replace('RT ', ' ').replace('&amp;', 'and')
# content = re.sub('[^A-Za-z0-9]+', ' ', content)
# content = content.lower()

# tokenized_word = word_tokenize(content)
# stop_words=set(stopwords.words("english"))
# filtered_sent=[]
# for w in tokenized_word:
#     if w not in stop_words:
#         filtered_sent.append(w)

li=list()
for i in df["named_ent"]:    
    te=i.split(",")    
    li.extend(te)

li = list(filter(None, li))

fdist = FreqDist(li)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

# Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=2, col=2)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=2, col=2)

'''
Plot the Geo-Distribution
'''
is_in_US=[]
geo = df[['user_location']]
df = df.fillna(" ")
for x in df['user_location']:
    check = False
    for s in STATES:
        if s in x:
            is_in_US.append(STATE_DICT[s] if s in STATE_DICT else s)
            check = True
            break
    if not check:
        is_in_US.append(None)

geo_dist = pd.DataFrame(is_in_US, columns=['State']).dropna().reset_index()
geo_dist = geo_dist.groupby('State').count().rename(columns={"index": "Number"}) \
        .sort_values(by=['Number'], ascending=False).reset_index()
geo_dist["Log Num"] = geo_dist["Number"].apply(lambda x: math.log(x, 2))


geo_dist['Full State Name'] = geo_dist['State'].apply(lambda x: INV_STATE_DICT[x])
geo_dist['text'] = geo_dist['Full State Name'] + '<br>' + 'Num: ' + geo_dist['Number'].astype(str)
fig.add_trace(go.Choropleth(
    locations=geo_dist['State'], # Spatial coordinates
    z = geo_dist['Log Num'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = "Blues",
    text=geo_dist['text'], # hover text
    showscale=False,
    geo = 'geo'
    ),
    row=1, col=2)

fig.update_layout(
    title_text= "Real-time tracking '{}' mentions on Twitter {} UTC".format(settings.TRACK_WORDS[0] ,datetime.datetime.utcnow().strftime('%m-%d %H:%M')),
    geo = dict(
        scope='usa',
    ),
    template="plotly_dark",
    margin=dict(r=20, t=50, b=50, l=20),
    annotations=[
        go.layout.Annotation(
            text="Source: Twitter",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ],
    showlegend=False,
    xaxis_rangeslider_visible=True
)

fig.show()

time.sleep(60)

Opened the database successfully


In [12]:
print(df["named_ent"])
li=list()
for i in df["named_ent"]:    
    te=i.split(",")    
    if len(te) > 0:
        for j in te:
            li.append(j)
    else:
        li.append(j)
li = list(filter(None, li))
print(li)

0                           
1                           
2           Mazi Nnamdi Kanu
3                           
4       dog Benji,Mottingham
               ...          
259    a Professional Styler
260                         
261     Newsmax,Mike Lindell
262                    House
263                         
Name: named_ent, Length: 264, dtype: object
['Mazi Nnamdi Kanu', 'dog Benji', 'Mottingham', 'Newsmax', 'Mike Lindell', 'Gizmo', 'Myanmar', 'Instaspa', 'Mike Lindell', 'Tw', 'GRAPHIC SPECIAL PLEA Shadow', 'HORRIFIC LEG INJURY', 'Radio Biafra App', 'Mark Nzuzukerberge', 'India', 'Mike Lindell', 'Brew Co Bucks', 'WellingtonMain Gym  ', 'Twitter', 'FacebookArrested', 'House', 'US House', 'Rep Greene', 'HallColdwell Banker CandC Prope', 'House', 'Manhattan', 'the US Embassy', 'Asmara', 'Twitter', 'Insta', 'Newsmax', 'Mike Lindell', 'ShadowPledge', 'Tumblr', 'Instagram to Yahoo', 'Elon Musk', 'Mark Zuckerberg', 'Katie', 'Mike Lindell', 'MB First Nations', 'AMC', 'Subs', 'SNS', '

In [11]:
content = ' '.join(df["named_ent"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

# Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=2, col=2)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=2, col=2)


3825

In [18]:
fdist = FreqDist(li)
# print(fdist)

# fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()
# print(fd)
# # Plot Bar chart   
fig.add_trace(go.Bar(x=fd["Word"], y=fd["Frequency"], name="Freq Dist"), row=2, col=2)
# 59, 89, 152
fig.update_traces(marker_color='rgb(59, 89, 152)', marker_line_color='rgb(8,48,107)', \
        marker_line_width=0.5, opacity=0.7, row=2, col=2)


AttributeError: 'Figure' object has no attribute 'add_trace'

In [19]:
import country_converter as coco
import pandas as np
import numpy as np
import sqlite3
import collections

conn1 = sqlite3.connect('twitter.db')
print("Opened the database successfully")

conn = conn1.cursor()

query = "SELECT * from  {}" .format(settings.TABLE_NAME)
df = pd.read_sql(query, con=conn1)

normal_names = df["user_location"].tolist()
iso2_codes = coco.convert(names=normal_names, to='ISO3')
print(iso2_codes)
counter=collections.Counter(iso2_codes)
print(counter)

Luzon  not found in regex
Africa  not found in regex
Opened the database successfully
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
None not found in regex
['TTO', 'ITA', 'ITA', 'MEX', 'ITA', 'USA', 'USA', 'ITA', 'ITA', 'USA', 'ITA', 'KOR', 'ITA', 'GRC', 'USA', 'CAN', 'GBR', 'ITA', 'USA', 'not found', 'not found', 'GBR', 'not found', 'ITA', 'USA', 'not found', 'USA', 'ITA', 'ARG', 'ZAF', 'not found', 'ITA', 'AUT', 'ITA', 'MYS', 'ITA', 'USA', 'ITA', 'ITA', 'ITA', 'GBR', 'CAN', 'USA', 'ITA', 'not found', 'IND', 'IND', 'IDN', 'ITA', 'USA', 'ARG', 'USA', 'ITA', 'I